# Testing Kits Production for Coronavirus (COVID-19) in the U.S.
By: Leonardo Calizaya

In [12]:
import numpy as np
import pandas as pd

D = list(range(1, 52)) #A list of numbers from 1 to 51 representing the 51 US states.
S = {1:"Center 1", 2: "Center 2", 3: "Center 3"} #A list of numbers from 1 to 3 representing the 3 distribution centers.

var_x = [(d, s) for d in D for s in S]
var_x_name = [f"x[{d}, {s}]" for (d, s) in var_x]

shipping_cost = 0.001 #Cost per mile to carry one testing kit.
production_cost = 1 #Cost to produce one testing kit.

center_capacity = {1: 1500000, 2: 1200000, 3: 1350000}

data = pd.read_excel("Project_Data2.xlsx", index_col = 0) #An excel sheet with the centers as column names and states a row indexes, each number represents the number of miles.

total_cost = pd.DataFrame(data[["State", "Center 1", "Center 2", "Center 3"]])
for center in S:
    total_cost[S[center]] = total_cost[S[center]]*shipping_cost + production_cost
    
print(total_cost)

                    State  Center 1  Center 2  Center 3
1                .Alabama     3.302     1.228     2.251
2                 .Alaska     3.933     6.221     6.780
3                .Arizona     1.530     3.016     3.904
4               .Arkansas     2.903     1.630     2.546
5             .California     1.000     3.529     4.372
6               .Colorado     1.979     2.602     3.393
7            .Connecticut     4.263     1.971     1.110
8               .Delaware     4.059     1.705     1.366
9   .District of Columbia     3.954     1.618     1.462
10               .Florida     3.644     1.292     2.221
11               .Georgia     3.529     1.000     2.070
12                .Hawaii     3.795     6.137     7.096
13                 .Idaho     1.614     3.276     3.980
14              .Illinois     3.109     1.658     2.263
15               .Indiana     3.329     1.526     2.046
16                  .Iowa     2.846     1.948     2.538
17                .Kansas     2.491     2.093   

In [13]:
total_cost.loc[17, 'Center 2'] = 2.2
from gurobipy import *

try:
    m = Model("mip1")
    
    x = m.addVars(var_x, vtype = GRB.CONTINUOUS, name = var_x_name)
    
    # (1) Each state must satisfy the demand of testing kits.
    cons1 = m.addConstrs(((sum(x[d, s] for s in S) >= data.loc[d, "Demand"]) for d in D))
    
    # (2) Each center has a maximum number of kits that they can produce.   
    
    cons2 = m.addConstrs(((sum(x[d, s] for d in D) <= center_capacity[s]) for s in S))
    
    m.setObjective(sum(sum(x[d, s]*total_cost.loc[d, S[s]] for s in S) for d in D))
    
    m.optimize()
    
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
    
    
except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 54 rows, 153 columns and 306 nonzeros
Model fingerprint: 0x1914a842
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+03, 2e+06]
Presolve time: 0.01s
Presolved: 54 rows, 153 columns, 306 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.087481e+06   0.000000e+00      0s
      59    4.7768268e+06   0.000000e+00   0.000000e+00      0s

Solved in 59 iterations and 0.01 seconds
Optimal objective  4.776826770e+06
x[1, 1] 0
x[1, 2] 47798
x[1, 3] 0
x[2, 1] 7103
x[2, 2] 0
x[2, 3] 0
x[3, 1] 63921
x[3, 2] 0
x[3, 3] 0
x[4, 1] 0
x[4, 2] 29160
x[4, 3] 0
x[5, 1] 372540
x[5, 2] 0
x[5, 3] 0
x[6, 1] 50292
x[6, 2] 0
x[6, 3] 0
x[7, 1] 0
x[7, 2] 0
x[7, 3] 35741
x[8, 1] 0
x[8, 2] 0
x[8, 3] 8980
x[9, 1] 0
x[9, 2] 0
x[9, 3] 6018
x[10, 1] 0
x[10, 2] 188014
x[10, 3] 0
x[11, 1] 0
x[11,

In [14]:
x[17, 2].Obj
x[17, 2].SAObjLow
x[17, 2].SAObjUp

inf

In [15]:
print(cons2[2].Pi)
print(cons2[2].CBasis)
print(cons2[2].SARHSLow)
print(cons2[2].SARHSUp)
print(cons2[2].RHS)
print(cons2[2].Slack)

-0.36599999999999966
-1
1059375.0
1310831.0
1200000.0
0.0


ANALYSIS

Using sensivity analysis from our previous result, we can discuss certain decisions that may be implemented in the testing kits productions for COVID-19. We will answer to some questions that will give us business insights.

1. How much the unit shipping cost from Kansas to the manufacturing center 2 can increase (assuming no change in the costs for the other states) before the current optimal solution would no longer be optimal?

To answer this question, we would need to look if the dual solution of our linear program changes if w

In [16]:
print(x[17,2].RC)
print(x[17,2].SAObjUp)

0.07499999999999973
inf


In [18]:
suma = 0
for a in m.getVars():
    suma += a.x
suma

3087481.0